
# 📓 CV PDF → CSV Extraction (Tutorial Version)

This notebook extracts structured information from **your 3 CV PDFs** and saves the results into a single CSV.  
It uses **pdfplumber** (no OCR) and includes enhanced parsing logic for LinkedIn/GitHub mentions that **don't** include full URLs.

**Output columns:**  
`["file_name","name","email","phone","linkedin","github","portfolio","other_links","raw_text"]`


## 1) Install & Import Libraries

In [24]:

%pip install -q pdfplumber pandas

import re
from pathlib import Path
import pandas as pd
import pdfplumber
from IPython.display import display, HTML


Note: you may need to restart the kernel to use updated packages.


## 2) Configuration (ONLY your 3 CV PDFs)

In [25]:
from pathlib import Path

# 👇 Your local folder
INPUT_DIR = Path(r"C:\Users\bbuser\Downloads")

# 👇 Your CV filenames
PDF_FILES = [
    "Duaa Hilal Al-Hashmi CV 2025.pdf",
    "Nusiba_Alnabhani_cv (1).pdf",
    "Janna Khalid Al Balushi Resume 14-09-25.pdf",
]

pdfs = [INPUT_DIR / f for f in PDF_FILES if (INPUT_DIR / f).exists()]
print(f"Found {len(pdfs)} CV PDFs:", [p.name for p in pdfs])


# Output CSV path
OUTPUT_CSV = str(INPUT_DIR / "cv_data_extracted_final.csv")

# Options
SAVE_RAW_TEXT = True
VERBOSE = True


Found 3 CV PDFs: ['Duaa Hilal Al-Hashmi CV 2025.pdf', 'Nusiba_Alnabhani_cv (1).pdf', 'Janna Khalid Al Balushi Resume 14-09-25.pdf']


## 3) Helper Functions (regex + extractors)

In [26]:
# Regex patterns
EMAIL_RE = re.compile(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b")
PHONE_RE = re.compile(r"""
    (?<!\w)(?:\+?\d{1,3}[\s\-\.]?)?(?:\(?\d{2,4}\)?[\s\-\.]?)\d{3,4}[\s\-\.]?\d{3,4}(?!\w)
""", re.VERBOSE)
LINK_RE = re.compile(r"https?://[^\s\)\]]+", re.IGNORECASE)

def clean_line(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def extract_text_from_pdf(path: str) -> str:
    """Extract text using pdfplumber (robust for typical CV formatting)."""
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text += page_text + "\n"
    return text

def guess_name_from_top(text: str, max_chars: int = 80) -> str:
    """Heuristic to guess full name from the top section of the CV."""
    lines = [clean_line(x) for x in text.splitlines() if clean_line(x)]
    for line in lines[:15]:
        if len(line) > max_chars:
            continue
        if EMAIL_RE.search(line) or LINK_RE.search(line) or PHONE_RE.search(line):
            continue
        # All-caps header like "DUAA HILAL AL-HASHMI"
        if line.isupper() and 1 <= len(line.split()) <= 5:
            return line.title()
        # Mostly letters & reasonable number of words
        letters_ratio = sum(ch.isalpha() or ch in " -.\'’" for ch in line) / max(1, len(line))
        words = line.split()
        if letters_ratio > 0.8 and 1 <= len(words) <= 5:
            return " ".join(w.capitalize() for w in words)
    return ""

def extract_linkedin(text: str, links: list[str]) -> str:
    # Prefer actual URLs first
    for l in links:
        if "linkedin.com" in l.lower():
            return l.strip()
    # Then pattern-based username capture
    m = re.search(r"(?i)linkedin\s*[:\-]?\s*([A-Za-z0-9\-\_\/\.]+)", text)
    if m:
        username = m.group(1).strip().strip('.').strip('/')
        if not username.startswith("http"):
            return f"https://www.linkedin.com/in/{username}"
    # If explicitly says LinkedIn Profile but no link
    if re.search(r"(?i)linkedin profile", text):
        return "LinkedIn Profile (no URL found)"
    return ""

def extract_github(text: str, links: list[str]) -> str:
    # Prefer actual URLs first
    for l in links:
        if "github.com" in l.lower():
            return l.strip()
    # Then pattern-based username capture
    m = re.search(r"(?i)github\s*[:\-]?\s*([A-Za-z0-9\-\_\/\.]+)", text)
    if m:
        username = m.group(1).strip().strip('.').strip('/')
        if not username.startswith("http"):
            return f"https://github.com/{username}"
    # If explicitly says GitHub Profile but no link
    if re.search(r"(?i)github profile", text):
        return "GitHub Profile (no URL found)"
    return ""


## 4) Sanity Check: Preview one CV's raw text (first 800 chars)

In [27]:

if pdfs:
    sample_text = extract_text_from_pdf(str(pdfs[0]))
    print(sample_text[:800])
else:
    print("No CV PDFs found. Check filenames and INPUT_DIR.")


Duaa Hilal Al-Hashmi
Al Batinah North | P: +968 91799668 | Duaahil44@gmail.com
EDUCATION
CODELINE
Data Science
Graduation Date: October 2025
SAUDI DIGITAL ACADEMY
FullStack Java Web Development Bootcamp
Graduation Date: December 2023
UNIVERSITY OF TECHNOLOGY AND APPLIED SCIENCES
Bachelor of Electronics and Telecommunications Engineering
Cumulative GPA: 3.2/4.0, Graduation Date: August 2022
EXPERIENCE
Telecommunications Regulatory Authority.
Muscat —Internship
● Completed the Nafath Training Program, acquiring expertise in programming languages (Java, JavaScript, Python, SQL),
frameworks (Spring Boot, Swagger-UI, Angular), and software design principles (RESTful APIs, SOLID)
● Applied skills in debugging tools (Postman, IntelliJ Debugger), IDEs (IntelliJ IDEA, Visual Studio Code), and compr


## 5) Extract Data From All 3 CV PDFs

In [28]:
rows = []
for p in pdfs:
    text = extract_text_from_pdf(str(p))

    if SAVE_RAW_TEXT:
        try:
            (p.parent / (p.stem + ".txt")).write_text(text, encoding="utf-8", errors="ignore")
        except Exception as e:
            if VERBOSE: print(f"[warn] cannot save raw text for {p.name}: {e}")

    emails = EMAIL_RE.findall(text)
    phones = PHONE_RE.findall(text)
    links = LINK_RE.findall(text)

    linkedin = extract_linkedin(text, links)
    github = extract_github(text, links)
    portfolio = next((u for u in links if any(k in u.lower() for k in ["portfolio","behance.net","dribbble.com","notion.site","about.me"])), "")
    other_links = [re.sub(r"[\)\]]+$", "", u) for u in links if u not in {linkedin, github, portfolio}]

    m = re.search(r"(?i)\bname\s*[:\-]\s*([^\n]+)", text)
    name = clean_line(m.group(1)) if m else guess_name_from_top(text)

    rows.append({
        "file_name": p.name,
        "name": name,
        "email": emails[0] if emails else "",
        "phone": phones[0] if phones else "",
        "linkedin": linkedin,
        "github": github,
        "portfolio": portfolio,
        "other_links": "; ".join(other_links[:8]),
        "raw_text": text
    })

df = pd.DataFrame(rows, columns=["file_name","name","email","phone","linkedin","github","portfolio","other_links","raw_text"])
print(f"Extracted {len(df)} row(s).")
df.head(10)


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


Extracted 3 row(s).


file_name  \
0             Duaa Hilal Al-Hashmi CV 2025.pdf   
1                  Nusiba_Alnabhani_cv (1).pdf   
2  Janna Khalid Al Balushi Resume 14-09-25.pdf   

                                 name                      email  \
0                Duaa Hilal Al-hashmi        Duaahil44@gmail.com   
1  Bachelor’s In Information Security  nusibaalnabhani@gmail.com   
2             Janna Khalid Al Balushi  janna.balushi@outlook.com   

           phone                                             linkedin  \
0  +968 91799668                                                        
1  +968-98995282  https://www.linkedin.com/in/com/in/nusiba-alnabhani   
2  +968 79141663                  https://www.linkedin.com/in/Profile   

                                                        github portfolio  \
0  https://github.com/Duaa3/Hotel-Managements-Project-Frontend             
1                                                                          
2                                   https://github.com/Profile             

                                                                                                                                                                                                                             other_links  \
0  https://github.com/Duaa3/Hotel-Managements-Project-Backend-; https://www.researchgate.net/publication/369452982_Date_fruit_classification_and_sorting_system_using_Artificial_; https://ijrar.org/viewfulltext.php?&p_id=IJRAR21D2332   
1                                                                                                                                                                                                                                          
2                                                                                                                                                                                                                                          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## 6) Display the Full DataFrame (scrollable)

In [29]:

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

html_table = df.to_html(escape=False, index=False)
display(HTML(f'''
<div style="max-height: 600px; overflow: auto; border: 1px solid #ccc; padding: 10px; white-space: nowrap;">
{html_table}
</div>
'''))


## 7) Save to CSV

In [30]:

df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ Data saved to: {OUTPUT_CSV}")


✅ Data saved to: C:\Users\bbuser\Downloads\cv_data_extracted_final.csv
